<a href="https://www.kaggle.com/code/bhavinmoriya/option-pricing-binomial-model?scriptVersionId=125326254" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

First step to do pricing of a options derivative is Binomial Model.

- Short call derivative and buy $\Delta$ units stock to evaluate the derivative price

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# 
def binomial_model(s0, s1u, s1d, K, r):
    u, d = s1u/s0, s1d/s0
    d1u, d1d = np.max([s1u-K,0]), np.max([s1d-K,0])
    delta = (d1u - d1d)/(s1u - s1d)
    
    p, q = (1+r-d)/(u-d), (u-1-r)/(u-d)
    d0 = (p * d1u/(1+r)) + (q * d1d/(1+r))
    print(f'No of stocks and Derivative price is : {delta, d0}')
    return delta, d0
    

In [3]:
binomial_model(50,65,40,55,.08)

No of stocks and Derivative price is : (0.4, 5.185185185185185)


(0.4, 5.185185185185185)

We could take advantage of this model to get the price of two step binomial model. 
- Apply BM to get from step 2 to 1
- Do the same from step 1 to 2.

In [4]:
def binomial_model1(u, d, d1u, d1d, r):
#     u, d = s1u/s0, s1d/s0
#     d1u, d1d = np.max([s1u-K,0]), np.max([s1d-K,0])
#     delta = (d1u - d1d)/(s1u - s1d)
    
    p, q = (1+r-d)/(u-d), (u-1-r)/(u-d)
    d0 = (p * d1u/(1+r)) + (q * d1d/(1+r))
#     print(f'Derivative price is : {d0}')
    return d0

In [5]:
d2pp,d2pm, d2mm = 25, 10, 5
u,d,r = 1.2, .9, .06
d1p, d1m = binomial_model1(u, d, d2pp,d2pm, r), binomial_model1(u, d, d2pm,d2mm, r)
binomial_model1(u, d, d1p, d1m, r)

11.728175309520989

In [6]:
# derivative price at 10 times step.
der_price = [np.sort(np.random.choice(range(1,50),size=11, replace=False))[::-1]]
while len(der_price[-1]) != 1:
    temp = []
    p = der_price[-1]
    k = len(p)
    for i in range(k-1):
        d1p, d1m = p[i], p[i+1]
        temp.append(binomial_model1(u, d, d1p, d1m, r))
    der_price.append(temp)
print(f'For a given price at 10 timestep : {der_price[0]},\nDerivative price is {der_price[-1][-1]}')

For a given price at 10 timestep : [49 37 32 30 27 21 20 18 10  4  3],
Derivative price is 13.335341544689204


One can calculate above price using binomial coefficient. Due to round off error we shall see that the values do not match.

In [7]:
from scipy.special import binom

In [8]:
coeff = np.array([binom(10,i) for i in range(11)])
p, q = (1+r-d)/(u-d), (u-1-r)/(u-d)
a = np.array([(p**i) * (q**(10-i)) for i in range(11)])
b = coeff * a
print(f'Derivative price is {b.dot(der_price[0])/((1+r)**10)}')

Derivative price is 12.0435667506742


In [9]:
def f(k):
    coeff = np.array([binom(k,i) for i in range(k+1)])
#     p, q = (1+r-d)/(u-d), (u-1-r)/(u-d)
    a = np.array([(p**i) * (q**(k-i)) for i in range(k+1)])
    b = coeff * a
#     print(coeff,a,b)
    print(f'Derivative price is {b.dot(der_price[10-k])/((1+r)**k)}')
    

In [10]:
for i in range(11):
    f(i)

Derivative price is 13.335341544689204
Derivative price is 13.206614120724613
Derivative price is 13.07793640756838
Derivative price is 12.949260233388362
Derivative price is 12.820533446384594
Derivative price is 12.691699269921953
Derivative price is 12.562695846189333
Derivative price is 12.433455985152133
Derivative price is 12.303907130776485
Derivative price is 12.173971550863037
Derivative price is 12.0435667506742


So far we go far the simulation is to the actual value of the option price that we derived earlier.